# Function 정의

## get_epl_season_data()

### 특정 시즌의 프리미어 리그 데이터를 transfermarkt 에서 가져오는 함수

- param season : 가져올 시즌 연도
- return : 해당 시즌의 팀 데이터 리스트

## scrape_transfermarkt()

### transfermarkt에서 여러 시즌의 프리미어 리그 데이터를 스크래핑하고 CSV 파일로 저장하는 함수

# Library Import

In [1]:
import os
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import random
from tqdm import tqdm

# get_epl_season_data 함수 정의

In [6]:
def get_epl_season_data(season):
    # transfermarkt 웹사이트 URL 구성
    url = f'https://www.transfermarkt.com/premier-league/startseite/wettbewerb/GB1/plus/?saison_id={season}'
    
    # 웹 스크래핑 탐지를 피하기 위한 헤더 설정
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36',
        'Accept-Language': 'en-US,en;q=0.9',
        'Accept-Encoding': 'gzip, deflate, br',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
        'Connection': 'keep-alive',
        'Upgrade-Insecure-Requests': '1'
    }
    # 웹 페이지 요청
    r = requests.get(url, headers=headers)
    
    # BeautifulSoup을 사용하여 HTML 파싱
    soup = BeautifulSoup(r.content, 'html.parser')
    
    # 팀 정보가 있는 테이블 행 추출 (프리미어 리그는 20개 팀만 경쟁)
    team_info = soup.find_all('tr', {'class': ['odd', 'even']})[:20]
    
    season_data = []
    for info in team_info:
        # 각 팀의 정보 추출
        team = info.find_all('td')
        season_data.append([
            season + 1,  # 시즌 (다음 해로 표시)
            team[1].text.strip().replace('\xa0', ''), # 팀 이름
            int(team[2].text.strip()),  # 스쿼드 크기
            float(team[3].text.strip()),  # 평균 나이
            team[5].text.strip()  # 평균 시장 가치
        ])
    # 과도한 요청을 보내지 않기 위해 1 ~ 3 초 사이의 랜덤한 시간 동안 대기
    time.sleep(random.uniform(1, 3))
    return season_data

# scrape_trasfermarkt 함수 정의

In [9]:
def scrape_transfermarkt():
    # 스크래핑할 시즌 범위 설정
    seasons = range(2004, 2024)
    all_team_data = []
    
    # 각 시즌의 데이터를 순차적으로 가져옴
    for season in tqdm(seasons, desc="get epl season data", unit="season"): # tqdm을 사용하여 진행 상황을 시각화
        season_data = get_epl_season_data(season)
        all_team_data.extend(season_data)
    
    # 수집된 데이터로 DataFrame 생성
    columns = ['Season', 'Club', 'Squad', 'Age', 'Market Value']
    df = pd.DataFrame(all_team_data, columns=columns)
    
    # 현재 작업 디렉토리 경로
    current_dir = os.getcwd()
    
    # 'raw_data' 폴더 경로
    raw_data_dir = os.path.join(current_dir, 'raw_data')
    # 'raw_data' 폴더가 없으면 생성
    os.makedirs(raw_data_dir, exist_ok=True)
    
    # CSV 파일 이름 설정
    output_file = 'epl_team_info_trasnfermarkt.csv'
    
    # 'raw_data' 폴더 안에 CSV 파일 저장 경로 설정
    output_path = os.path.join(raw_data_dir, output_file)
    
    # DataFrame을 CSV 파일로 저장
    df.to_csv(output_path, index=False)
    
    print(f'{output_file} 데이터가 {output_path}에 저장되었습니다.')

# 사용 예시

In [10]:
if __name__ == "__main__":
    scrape_transfermarkt()

get epl season data: 100%|██████████████████| 20/20 [02:44<00:00,  8.24s/season]

epl_team_info_trasnfermarkt.csv 데이터가 /Users/sonjeehyung/Desktop/Project/승부예측/raw_data/epl_team_info_trasnfermarkt.csv에 저장되었습니다.
